<a href="https://colab.research.google.com/github/ekansh2004/Machine-Learning-Projects/blob/main/KnnMedAbv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install anvil-uplink

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.9/95.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 2.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
import anvil.server
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
file_path = "/content/drive/My Drive/final_train.csv"
data = pd.read_csv(file_path)

In [ ]:
data.head()

,LABEL,ABV
0,transverse aortic constriction,TAC
1,hodgkins lymphoma,HD
2,methicillinsusceptible s aureus,MSSA
3,parathyroid hormonerelated protein,PTHrP
4,lipoperoxidation,LPO


In [ ]:
labels = data['LABEL']
abvs = data['ABV']

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(abvs)

y = labels

In [ ]:
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X, y)

KNeighborsClassifier(n_neighbors=1)

In [ ]:
def search_abbreviation(abbreviation):
    abv_vector = vectorizer.transform([abbreviation])

    predicted_label = knn.predict(abv_vector)

    return predicted_label[0]

In [ ]:
anvil.server.connect("server_5RI2BHPKB7ZYRLZH3NTW5TY7-67ZYMSMSIU4WLEQT")

@anvil.server.callable
def get_answer(abbreviation):
    return search_abbreviation(abbreviation)


print("Connected to Anvil server and callable function is ready!")
anvil.server.wait_forever()

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default Environment" as SERVER
Connected to Anvil server and callable function is ready!


In [ ]:
from sklearn.metrics import accuracy_score

X_test = X[:1000]
y_test = y[:1000]
y_pred = knn.predict(X_test)

training_accuracy = accuracy_score(y_test, y_pred)

print(f"Training accuracy: {training_accuracy * 100:.2f}%")
print(f"Training error: {(1 - training_accuracy) * 100:.2f}%")


Training accuracy: 80.80%
Training error: 19.20%


In [ ]:
from sklearn.model_selection import cross_val_score
import numpy as np

k_values = range(1, 20)
cv_scores = []

for k in k_values:
    knn = KNeighborsClassifier(n_neighbors=k)
    scores = cross_val_score(knn, X, y, cv=5, scoring='accuracy')
    cv_scores.append(scores.mean())

optimal_k = k_values[np.argmax(cv_scores)]
print("The optimal number of neighbors is:", optimal_k)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklea

The optimal number of neighbors is: 1
